In [1]:
import pandas as pd 
import numpy as np 
import nltk
import re
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall
import gensim

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dropout, Dense, Embedding
from keras import Sequential

import preprocess_kgptalkie as ps

  Using cached spacy-2.2.3.tar.gz (5.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [49 lines of output]
      Traceback (most recent call last):
        File "/tmp/pip-build-env-qjlh6hpm/overlay/lib/python3.9/site-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = parse_requirement(requirement_string)
        File "/tmp/pip-build-env-qjlh6hpm/overlay/lib/python3.9/site-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
        File "/tmp/pip-build-env-qjlh6hpm/overlay/lib/python3.9/site-packages/setuptools/_vendor/packaging/_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
        File "/tmp/pip-

2023-04-01 11:17:44.687131: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-01 11:17:44.689008: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-01 11:17:44.725956: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-01 11:17:44.726611: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 11:17:45.221498: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
fake = pd.read_csv('news_articles.csv')
fake['label'] = fake['label'].replace({'Real':1,'Fake':0})
fake = fake.dropna(axis=0)
fake.isna().sum()

author                     0
published                  0
title                      0
text                       0
language                   0
site_url                   0
main_img_url               0
type                       0
label                      0
title_without_stopwords    0
text_without_stopwords     0
hasImage                   0
dtype: int64

In [3]:
from newspaper import Article
import pandas as pd


array = ['https://www.timesnownews.com/viral/florida-man-thief-steals-tequila-from-liquor-store-in-cape-coral-article-96661579']

titles = []
texts = []
dates = []

for url in fake['site_url']:
    article = Article(url,language= 'en')

    article.download()
    article.parse()

    titles.append(article.title)
    texts.append(article.text)
    dates.append(article.publish_date)

data = pd.DataFrame({'title':titles,'text':texts,'date':dates})


ArticleException: Article `download()` failed with No connection adapters were found for ':/100percentfedup.com' on URL :/100percentfedup.com

In [11]:
df = pd.read_csv('preprocessed_data.csv')
x = [row.split() for row in df.text.tolist()]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

In [12]:
def preprocess_data(x_test):
    unknown_pub = []
    for idx, row in enumerate(x_test.text.values):
        try:
            record = row.split('-', maxsplit=1)
            assert(len(record[0]) < 120)
        except:
            unknown_pub.append(idx)
    publisher = []
    tmp_text = []
    for idx, row in enumerate(x_test.text.values):
        if idx in unknown_pub:
            tmp_text.append(row)
            publisher.append('unknown')
        else:
            tmp_text.append(row)
            publisher.append('unknown')
    x_test['publisher'] = publisher
    x_test.text = tmp_text
    x_test.text = x_test.title + " " + x_test.text
    x_test.text = x_test.text.apply(lambda x:str(x).lower())
    x_test.text.apply(lambda x:ps.remove_special_chars(x))
    x_test_preprocessed = [row.split() for row in x_test.text.tolist()]
    x_test_preprocessed = tokenizer.texts_to_sequences(x_test_preprocessed)
    x_test_preprocessed = pad_sequences(x_test_preprocessed, maxlen = 1000)
    return x_test_preprocessed

In [13]:
import tensorflow as tf

model = tf.keras.models.load_model('saved_model')
model.summary()

2023-04-01 11:18:52.929237: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-04-01 11:18:52.932117: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-04-01 11:18:52.932199: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 100)         37537400  
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 37,687,929
Trainable params: 150,529
Non-trainable params: 37,537,400
_________________________________________________________________


2023-04-01 11:18:54.005115: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-01 11:18:54.006343: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-01 11:18:54.007192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [14]:
test_data = {'title': ['New York Man Wanted For Infecting 240 Men And Women With HIV!!!'], 'text': ["Hello I am drowning under the water help me googoogaga-"]}
temp = pd.DataFrame(fake[['title','text']])

In [15]:
temp.head()

,title,text
0,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...
1,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...
2,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...
3,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...
4,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...


In [16]:
a = preprocess_data(temp)

In [17]:
a


array([[    0,     0,     0, ...,   377,    49,  1186],
       [    0,     0,     0, ...,  2544,    49,  1186],
       [    0,     0,     0, ...,     7,    49,  1186],
       ...,
       [    0,     0,     0, ...,    93,   125,  2078],
       [    0,     0,     0, ..., 15223,   348,   278],
       [    0,     0,     0, ...,    72,   174,   314]], dtype=int32)

In [18]:
c = (model.predict(a) >= 0.5).astype(int)

2023-04-01 11:19:07.249200: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-01 11:19:07.250382: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-01 11:19:07.251121: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

64/64 [==============================] - 7s 108ms/step


In [19]:
from sklearn.metrics import accuracy_score

accuracy_score(fake['label'],c)

0.6425427872860636

In [20]:
fake.isna().sum()

author                     0
published                  0
title                      0
text                       0
language                   0
site_url                   0
main_img_url               0
type                       0
label                      0
title_without_stopwords    0
text_without_stopwords     0
hasImage                   0
dtype: int64